In [336]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import time
from sklearn.metrics import accuracy_score
from collections import Counter
import torch

from deep_logic.utils.base import validate_network, set_seed, tree_to_formula
from deep_logic.utils.relunn import get_reduced_model, prune_features
from deep_logic.utils.sigmoidnn import prune_equal_fanin
from deep_logic import logic

In [74]:
df = pd.read_csv('data/breast/METABRIC_RNA_Mutation.csv')

c:\users\pietr\anaconda3\envs\deep-logic\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (678,688,690,692) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [93]:
df.iloc[:, :10]

,patient_id,age_at_diagnosis,type_of_breast_surgery,cancer_type,cancer_type_detailed,cellularity,chemotherapy,pam50_+_claudin-low_subtype,cohort,er_status_measured_by_ihc
0,0,75.65,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,NaN,0,claudin-low,1.0,Positve
1,2,43.19,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumA,1.0,Positve
2,5,48.87,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,1,LumB,1.0,Positve
3,6,47.68,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Moderate,1,LumB,1.0,Positve
4,8,76.97,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,1,LumB,1.0,Positve
...,...,...,...,...,...,...,...,...,...,...
1899,7295,43.10,BREAST CONSERVING,Breast Cancer,Breast Invasive Lobular Carcinoma,High,0,LumA,4.0,Positve
1900,7296,42.88,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumB,4.0,Positve
1901,7297,62.90,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumB,4.0,Positve
1902,7298,61.16,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,0,LumB,4.0,Positve


In [95]:
df.iloc[:, 10:20]

,er_status,neoplasm_histologic_grade,her2_status_measured_by_snp6,her2_status,tumor_other_histologic_subtype,hormone_therapy,inferred_menopausal_state,integrative_cluster,primary_tumor_laterality,lymph_nodes_examined_positive
0,Positive,3.0,NEUTRAL,Negative,Ductal/NST,1,Post,4ER+,Right,10.0
1,Positive,3.0,NEUTRAL,Negative,Ductal/NST,1,Pre,4ER+,Right,0.0
2,Positive,2.0,NEUTRAL,Negative,Ductal/NST,1,Pre,3,Right,1.0
3,Positive,2.0,NEUTRAL,Negative,Mixed,1,Pre,9,Right,3.0
4,Positive,3.0,NEUTRAL,Negative,Mixed,1,Post,9,Right,8.0
...,...,...,...,...,...,...,...,...,...,...
1899,Positive,3.0,NEUTRAL,Negative,Lobular,1,Pre,3,Right,1.0
1900,Positive,3.0,GAIN,Positive,Ductal/NST,0,Pre,5,NaN,1.0
1901,Positive,3.0,NEUTRAL,Negative,Ductal/NST,1,Post,1,Left,45.0
1902,Positive,2.0,NEUTRAL,Negative,Ductal/NST,1,Post,1,NaN,12.0


In [96]:
df.iloc[:, 20:31]

,mutation_count,nottingham_prognostic_index,oncotree_code,overall_survival_months,overall_survival,pr_status,radio_therapy,3-gene_classifier_subtype,tumor_size,tumor_stage,death_from_cancer
0,NaN,6.044,IDC,140.500000,1,Negative,1,ER-/HER2-,22.0,2.0,Living
1,2.0,4.020,IDC,84.633333,1,Positive,1,ER+/HER2- High Prolif,10.0,1.0,Living
2,2.0,4.030,IDC,163.700000,0,Positive,0,NaN,15.0,2.0,Died of Disease
3,1.0,4.050,MDLC,164.933333,1,Positive,1,NaN,25.0,2.0,Living
4,2.0,6.080,MDLC,41.366667,0,Positive,1,ER+/HER2- High Prolif,40.0,2.0,Died of Disease
...,...,...,...,...,...,...,...,...,...,...,...
1899,4.0,5.050,ILC,196.866667,1,Positive,1,ER+/HER2- Low Prolif,25.0,NaN,Living
1900,6.0,5.040,IDC,44.733333,0,Negative,1,NaN,20.0,NaN,Died of Disease
1901,4.0,6.050,IDC,175.966667,0,Positive,1,NaN,25.0,NaN,Died of Disease
1902,15.0,5.050,IDC,86.233333,0,Positive,0,ER+/HER2- High Prolif,25.0,NaN,Died of Other Causes


In [79]:
columns = df.columns[:31]
columns

Index(['patient_id', 'age_at_diagnosis', 'type_of_breast_surgery',
       'cancer_type', 'cancer_type_detailed', 'cellularity', 'chemotherapy',
       'pam50_+_claudin-low_subtype', 'cohort', 'er_status_measured_by_ihc',
       'er_status', 'neoplasm_histologic_grade',
       'her2_status_measured_by_snp6', 'her2_status',
       'tumor_other_histologic_subtype', 'hormone_therapy',
       'inferred_menopausal_state', 'integrative_cluster',
       'primary_tumor_laterality', 'lymph_nodes_examined_positive',
       'mutation_count', 'nottingham_prognostic_index', 'oncotree_code',
       'overall_survival_months', 'overall_survival', 'pr_status',
       'radio_therapy', '3-gene_classifier_subtype', 'tumor_size',
       'tumor_stage', 'death_from_cancer'],
      dtype='object')

In [85]:
Counter(df['death_from_cancer'])

Counter({'Living': 801,
         'Died of Disease': 622,
         'Died of Other Causes': 480,
         nan: 1})

In [86]:
Counter(df['overall_survival'])

Counter({1: 801, 0: 1103})

# Task

In [370]:
y = df['overall_survival']
Counter(y)

Counter({1: 801, 0: 1103})

# Concepts

In [218]:
def transform(x, n_bins=3, encoding=True, binarize=True, strategy='uniform'):
    xnp = x.values
    enc = LabelEncoder()
    kbin = KBinsDiscretizer(n_bins=n_bins, encode='onehot-dense', strategy=strategy)
    if encoding:
        xnp = enc.fit_transform(xnp)
    if binarize:
        xnp = kbin.fit_transform(xnp.reshape(-1, 1))
    return xnp

In [258]:
df['mutation_count']

0        NaN
1        2.0
2        2.0
3        1.0
4        2.0
        ... 
1899     4.0
1900     6.0
1901     4.0
1902    15.0
1903     3.0
Name: mutation_count, Length: 1904, dtype: float64

In [282]:
Counter(df['type_of_breast_surgery'])

Counter({'MASTECTOMY': 1127, 'BREAST CONSERVING': 755, nan: 22})

In [275]:
concept_list = []
concept_list.append(transform(df['age_at_diagnosis'], n_bins=5, encoding=False, strategy='quantile'))
concept_list.append(transform(df['type_of_breast_surgery'], n_bins=3))
concept_list.append(transform(df['cellularity'], n_bins=4))
concept_list.append(df['chemotherapy'].values.reshape(-1, 1))
concept_list.append(transform(df['pam50_+_claudin-low_subtype'], n_bins=7, binarize=True))
concept_list.append(transform(df['er_status'], binarize=False).reshape(-1, 1))
concept_list.append(transform(df['neoplasm_histologic_grade'], n_bins=4))
z = df['tumor_other_histologic_subtype'].replace('Other', np.NaN).replace('Metaplastic', np.NaN)
concept_list.append(transform(z, n_bins=7))
concept_list.append(transform(df['inferred_menopausal_state'], binarize=False).reshape(-1, 1))
concept_list.append(transform(df['primary_tumor_laterality'], n_bins=3))
concept_list.append(transform(df['lymph_nodes_examined_positive'], n_bins=5, encoding=False, strategy='quantile'))
concept_list.append(transform(df['mutation_count'], n_bins=5, strategy='quantile'))
concept_list.append(transform(df['pr_status'], binarize=False).reshape(-1, 1))
concept_list.append(transform(df['radio_therapy'], binarize=False).reshape(-1, 1))
concept_list.append(transform(df['tumor_stage'], n_bins=4))
concept_list[-1].sum(axis=0)

c:\users\pietr\anaconda3\envs\deep-logic\lib\site-packages\sklearn\preprocessing\_discretization.py:220: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '


array([479., 800., 115., 510.])

In [418]:
raw_concepts = ['age_at_diagnosis', 'type_of_breast_surgery',
            'cellularity', 'chemotherapy',
            'pam50_+_claudin-low_subtype',
            'er_status', 'neoplasm_histologic_grade',
            'tumor_other_histologic_subtype',
            'inferred_menopausal_state',
            'primary_tumor_laterality', 'lymph_nodes_examined_positive',
            'mutation_count', 'pr_status',
            'radio_therapy',
            'tumor_stage']
len(raw_concepts)

15

In [422]:
clinic_concepts = []
for clist, cname in zip(concept_list, raw_concepts):
    if clist.shape[1] > 1:
        for i in range(clist.shape[1]):
            clinic_concepts.append(f'{cname}_{i}')
    else:
        clinic_concepts.append(f'{cname}')
clinic_concepts

['age_at_diagnosis_0',
 'age_at_diagnosis_1',
 'age_at_diagnosis_2',
 'age_at_diagnosis_3',
 'age_at_diagnosis_4',
 'type_of_breast_surgery_0',
 'type_of_breast_surgery_1',
 'type_of_breast_surgery_2',
 'cellularity_0',
 'cellularity_1',
 'cellularity_2',
 'cellularity_3',
 'chemotherapy',
 'pam50_+_claudin-low_subtype_0',
 'pam50_+_claudin-low_subtype_1',
 'pam50_+_claudin-low_subtype_2',
 'pam50_+_claudin-low_subtype_3',
 'pam50_+_claudin-low_subtype_4',
 'pam50_+_claudin-low_subtype_5',
 'pam50_+_claudin-low_subtype_6',
 'er_status',
 'neoplasm_histologic_grade_0',
 'neoplasm_histologic_grade_1',
 'neoplasm_histologic_grade_2',
 'neoplasm_histologic_grade_3',
 'tumor_other_histologic_subtype_0',
 'tumor_other_histologic_subtype_1',
 'tumor_other_histologic_subtype_2',
 'tumor_other_histologic_subtype_3',
 'tumor_other_histologic_subtype_4',
 'tumor_other_histologic_subtype_5',
 'tumor_other_histologic_subtype_6',
 'inferred_menopausal_state',
 'primary_tumor_laterality_0',
 'primary

In [286]:
c = np.hstack(concept_list)
c.shape

(1904, 50)

In [287]:
clf = DecisionTreeClassifier()
cross_val_score(clf, c, y, cv=10)

array([0.59685864, 0.59162304, 0.47120419, 0.53926702, 0.57894737,
       0.64736842, 0.62631579, 0.57368421, 0.61578947, 0.65263158])

In [315]:
clf = RandomForestClassifier()
cross_val_score(clf, c, y, cv=10)

array([0.68586387, 0.68586387, 0.43979058, 0.62303665, 0.67368421,
       0.66842105, 0.62105263, 0.64210526, 0.72105263, 0.65263158])

In [427]:
x = df.iloc[:, 31:173].values
x.shape

(1904, 142)

In [426]:
rna_concepts = list(df.columns[31:173])
len(rna_concepts)

142

In [411]:
z = MinMaxScaler().fit_transform(np.hstack([c, x]))
z.shape

(1904, 192)

In [412]:
clf = DecisionTreeClassifier()
cross_val_score(clf, z, y, cv=10)

array([0.59162304, 0.64397906, 0.42931937, 0.55497382, 0.55789474,
       0.47894737, 0.56842105, 0.57368421, 0.56842105, 0.62631579])

In [413]:
clf = RandomForestClassifier()
cross_val_score(clf, z, y, cv=10)

array([0.66492147, 0.69633508, 0.47120419, 0.67539267, 0.65789474,
       0.64210526, 0.62105263, 0.62631579, 0.67368421, 0.63684211])

In [430]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
concepts = clinic_concepts + rna_concepts
n_rep = 10
tot_epochs = 5001
prune_epochs = 2001
len(concepts)

192

In [435]:
def train_nn(x_train, y_train, need_pruning, seed, device, relu=False):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device)
    layers = [
        torch.nn.Linear(x_train.size(1), 500),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(500, 100),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(100, 50),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(50, 10),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(10, 2),
        torch.nn.Softmax(dim=1),
    ]
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
    loss_form = torch.nn.CrossEntropyLoss()
    model.train()
    for epoch in range(tot_epochs):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train)
        # Compute Loss
        loss = loss_form(y_pred, y_train)

        for module in model.children():
            if isinstance(module, torch.nn.Linear):
                loss += 0.001 * torch.norm(module.weight, 1)
                loss += 0.001 * torch.norm(module.bias, 1)
                break

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch > prune_epochs and need_pruning:
            prune_features(model, n_classes=1, device=device)
            need_pruning = False
            
        # compute accuracy
        if epoch % 500 == 0:
            y_pred_d = torch.argmax(y_pred, dim=1)
            accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
            print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

In [436]:
need_pruning = True
method = 'pruning'
methods = []
splits = []
explanations = []
explanations_inv = []
model_accuracies = []
explanation_accuracies = []
explanation_accuracies_inv = []
elapsed_times = []
elapsed_times_inv = []

n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)

for split, (train_index, test_index) in enumerate(skf.split(z, y)):
    print(f'Split [{split+1}/{n_splits}]')
    x_train, x_test = torch.FloatTensor(z[train_index]), torch.FloatTensor(z[test_index])
    y_train, y_test = torch.LongTensor(y.values[train_index]), torch.LongTensor(y.values[test_index])
    
#     if split not in [5]: continue
    
    model = train_nn(x_train, y_train, need_pruning, split, device)
    
    y_preds = model(x_test.to(device)).cpu().detach().numpy()
    model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds.argmax(axis=1))
    print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
    
    # positive class
    target_class = 1
    start = time.time()
    global_explanation, _, counter = logic.relunn.combine_local_explanations(model, 
                                                                       x_train.to(device), y_train.to(device), 
                                                                       target_class=target_class,
                                                                       topk_explanations=3,
                                                                       method=method, device=device)
    elapsed_time = time.time() - start
    if global_explanation:
        explanation_accuracy, _ = logic.base.test_explanation(global_explanation, target_class, x_test, y_test)
        explanation = logic.base.replace_names(global_explanation, concepts)
    print(f'\t Class {target_class} - Global explanation: "{explanation}" - Accuracy: {explanation_accuracy:.4f}')
    print(f'\t Elapsed time {elapsed_time}')
        
    # negative class
    target_class = 0
    start = time.time()
    global_explanation_inv, _, counter_inv = logic.relunn.combine_local_explanations(model, 
                                                                           x_train.to(device), y_train.to(device), 
                                                                           target_class=target_class,
                                                                           topk_explanations=3,
                                                                           method=method, device=device)
    elapsed_time_inv = time.time() - start
    if global_explanation_inv:
        explanation_accuracy_inv, _ = logic.base.test_explanation(global_explanation_inv, target_class, x_test, y_test)
        explanation_inv = logic.base.replace_names(global_explanation_inv, concepts)
    print(f'\t Class {target_class} - Global explanation: "{explanation_inv}" - Accuracy: {explanation_accuracy_inv:.4f}')
    print(f'\t Elapsed time {elapsed_time_inv}')
    
    methods.append(method)
    splits.append(split)
    explanations.append(explanation)
    explanations_inv.append(explanation_inv)
    model_accuracies.append(model_accuracy)
    explanation_accuracies.append(explanation_accuracy)
    explanation_accuracies_inv.append(explanation_accuracy_inv)
    elapsed_times.append(elapsed_time)
    elapsed_times_inv.append(elapsed_time_inv)

Split [1/10]
	 Epoch 0: train accuracy: 0.4203
	 Epoch 500: train accuracy: 0.8797
	 Epoch 1000: train accuracy: 0.9066
	 Epoch 1500: train accuracy: 0.9107
	 Epoch 2000: train accuracy: 0.9154
	 Epoch 2500: train accuracy: 0.6556
	 Epoch 3000: train accuracy: 0.6614
	 Epoch 3500: train accuracy: 0.6684
	 Epoch 4000: train accuracy: 0.6708
	 Epoch 4500: train accuracy: 0.6713
	 Epoch 5000: train accuracy: 0.6696
	 Model's accuracy: 0.6178
	 Class 1 - Global explanation: "(age_at_diagnosis_0 & inferred_menopausal_state & ~age_at_diagnosis_4 & ~pam50_+_claudin-low_subtype_1 & ~neoplasm_histologic_grade_1 & ~tumor_other_histologic_subtype_1 & ~primary_tumor_laterality_2 & ~mutation_count_3) | (~age_at_diagnosis_0 & ~age_at_diagnosis_4 & ~pam50_+_claudin-low_subtype_1 & ~tumor_other_histologic_subtype_1 & ~inferred_menopausal_state & ~primary_tumor_laterality_2 & ~mutation_count_3 & ~bmp2)" - Accuracy: 0.5707
	 Elapsed time 1.8749885559082031
	 Class 0 - Global explanation: "(age_at_diagno

KeyboardInterrupt: 